In [2]:
import pandas as pd

In [3]:
table_url = 'https://www.mapsofworld.com/thematic-maps/types-of-governments.html'
tables = pd.read_html(table_url)
tables

[            0
 0  Disclaimer,
                                                    0  \
 0  CloseDisclaimer : All efforts have been made t...   
 
                                                    1  
 0  CloseDisclaimer : All efforts have been made t...  ,
                                                    0  \
 0  ColorGovernment Type Transitional government F...   
 
                                                    1  
 0  ColorGovernment Type Communist state Democrati...  ,
     Color                                    Government Type
 0     NaN                            Transitional government
 1     NaN                                         Federation
 2     NaN                        Federal democratic Republic
 3     NaN     Constitutional monarchy and commonwealth realm
 4     NaN  Federal Parliamentary democracy and a Commonwe...
 5     NaN                 Democratic Constitutional Republic
 6     NaN                             Constitutional emirate
 7     NaN     

In [4]:
gov_df = tables[5]
gov_df.columns = ['Country', 'Govt Type', 'Head of State', 'Head of Government']
new_gov_df = gov_df.rename(columns={'Govt Type':'Government_Type', 'Head of State':'Head_of_State', 'Head of Government':'Head_of_Government'})

gov_df.set_index('Country', inplace=True)
gov_df

,Govt Type,Head of State,Head of Government
Country,,,
Afghanistan,Presidential Islamic republic,President - Ashraf Ghani,Chief Executive Officer - Abdullah Abdullah
Albania,Parliamentary republic,President - Bujar Nishani,Prime Minister - Edi Rama
Algeria,Presidential republic,President - Abdelaziz Bouteflika,Prime Minister - Abdelmalek Sellal
Andorra,Unitary parliamentary Semi-Elective diarchy,Episcopal Co-Prince - Joan Enric Vives Sicília...,Head of Government - Antoni Martí
Angola,Presidential republic,President - Jose Eduardo dos Santos,NaN
...,...,...,...
Venezuela,Federal presidential republic,President - Nicolas Maduro,NaN
Vietnam,Communist state,President - Tran Dại Quang,Prime Minister - Nguyen Xuan Phúc
Yemen,in transition (Provisional government),President - Abdrabbuh Mansur Hadi,Prime Minister - Ahmed Obeid bin Daghr
